In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, EfficientNetB0, resnet50, efficientnet
from tensorflow.keras.optimizers import Adam

# ---------------------------------------------------
# 1. Load the TF-Flowers Dataset
# ---------------------------------------------------
# Create train/validation split
(dataset_train, dataset_test), info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

# Number of classes in TF-Flowers
num_classes = info.features['label'].num_classes

# ---------------------------------------------------
# 2. Preprocess Data for ResNet and EfficientNet
# ---------------------------------------------------
IMG_SIZE = 224
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_resnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = resnet50.preprocess_input(image)
    return image, tf.one_hot(label, num_classes)

def preprocess_efficientnet(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = efficientnet.preprocess_input(image)
    return image, tf.one_hot(label, num_classes)

# Create tf.data pipelines
train_ds_resnet = (dataset_train
                   .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                   .batch(batch_size)
                   .prefetch(AUTOTUNE))

test_ds_resnet = (dataset_test
                  .map(preprocess_resnet, num_parallel_calls=AUTOTUNE)
                  .batch(batch_size)
                  .prefetch(AUTOTUNE))

train_ds_eff = (dataset_train
                .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
                .batch(batch_size)
                .prefetch(AUTOTUNE))

test_ds_eff = (dataset_test
               .map(preprocess_efficientnet, num_parallel_calls=AUTOTUNE)
               .batch(batch_size)
               .prefetch(AUTOTUNE))

# ---------------------------------------------------
# 3. Define ResNet50 Model without Dynamic Gating
# ---------------------------------------------------
def create_resnet_model(input_shape, num_classes):
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = True

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="ResNet")
    return model

# ---------------------------------------------------
# 4. Define EfficientNetB0 Model without Dynamic Gating
# ---------------------------------------------------
def create_efficientnet_model(input_shape, num_classes):
    base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = True

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs=base_model.input, outputs=outputs, name="EfficientNet")
    return model

# ---------------------------------------------------
# 5. Compile and Train the Models
# ---------------------------------------------------
# ResNet50 Model
resnet_model = create_resnet_model((IMG_SIZE, IMG_SIZE, 3), num_classes)
resnet_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning ResNet50 on TF-Flowers ---")
history_resnet = resnet_model.fit(train_ds_resnet, epochs=50, validation_data=test_ds_resnet, verbose=1)

# EfficientNetB0 Model
eff_model = create_efficientnet_model((IMG_SIZE, IMG_SIZE, 3), num_classes)
eff_model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Fine-Tuning EfficientNetB0 on TF-Flowers ---")
history_eff = eff_model.fit(train_ds_eff, epochs=50, validation_data=test_ds_eff, verbose=1)

# Evaluate ResNet50
loss_resnet, acc_resnet = resnet_model.evaluate(test_ds_resnet, verbose=0)
print(f"ResNet50 Final Accuracy: {acc_resnet:.4f}")

# Evaluate EfficientNetB0
loss_eff, acc_eff = eff_model.evaluate(test_ds_eff, verbose=0)
print(f"EfficientNetB0 Final Accuracy: {acc_eff:.4f}")

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

--- Fine-Tuning ResNet50 on TF-Flowers ---
Epoch 1/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 139s 699ms/step - accuracy: 0.7121 - loss: 0.7832 - val_accuracy: 0.8978 - val_loss: 0.4790
Epoch 2/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 26s 287ms/step - accuracy: 0.9710 - loss: 0.0925 - val_accuracy: 0.9428 - val_loss: 0.1968
Epoch 3/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 289ms/step - accuracy: 0.9983 - loss: 0.0133 - val_accuracy: 0.9537 - val_loss: 0.1566
Epoch 4/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 42s 300ms/step - accuracy: 0.9997 - loss: 0.0026 - val_accuracy: 0.9591 - val_loss: 0.1520
Epoch 5/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 41s 300ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.9591 - val_loss: 0.1567
Epoch 6/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 28s 304ms/step - accuracy: 1.0000 - loss: 7.0212e-04 - val_accuracy: 0.9591 - val_loss: 0.15